# State

State can be changed; the only thing that the jury is out on is who triggers the state change.
There are, fundamentally, two ways:
 - States are actual classes with behaviors, and these behaviors switch the actual state from one to another.
 - States and transitions are just enumerations. We have a special component called a state machine that
performs the actual transitions.

## state-driven transition

In [2]:
# trigger

class LightSwitch:
    def __init__(self):
        self.state = OffState()
    def set_state(self, state):
        self.state = state
    def on(self):
        self.state.on(self)
    def off(self):
        self.state.off(self)

In [3]:
# state class

class State:
    def on(self, switch: LightSwitch):
        print("Light is already on")
    def off(self, switch: LightSwitch):
        print("Light is already off")

class OnState(State):
    def __init__(self):
        print("Light turned on")
    def off(self, switch):
        print("Switching Light off...")
        switch.set_state(OffState())
        del self

class OffState(State):
    def __init__(self):
        print("Light turned off")
    def on(self, switch):
        print("Switching Light on...")
        switch.set_state(OnState())
        del self

In [4]:
ls = LightSwitch()
ls.on()
ls.off()
ls.off()

Light turned off
Switching Light on...
Light turned on
Switching Light off...
Light turned off
Light is already off


## Event-driven Transition

The trigger of the state change is user controlled.

In [7]:
import sys
  
class States(object):
    def __init__(self):
        self.done = False
        self.next = None
        self.quit = False
        self.previous = None
  
class Menu(States):
    def __init__(self):
        States.__init__(self)
        self.next = 'game'
    def cleanup(self):
        print('cleaning up Menu state stuff')
    def startup(self):
        print('starting Menu state stuff')
    def get_event(self, event):
        # if event.type == pg.KEYDOWN:
        #     print('Menu State keydown')
        # elif event.type == pg.MOUSEBUTTONDOWN:
        #     self.done = True
        # dummy event
        self.done = True
    def update(self, screen, dt):
        self.draw(screen)
    def draw(self, screen):
        print("draw menu")
  
class Gameplay(States):
    def __init__(self):
        States.__init__(self)
        self.next = 'menu'
    def cleanup(self):
        print('cleaning up Game state stuff')
    def startup(self):
        print('starting Game state stuff')
    def get_event(self, event):
        # if event.type == pg.KEYDOWN:
        #     print('Game State keydown')
        # elif event.type == pg.MOUSEBUTTONDOWN:
        #     self.done = True
        # dummy event
        self.done = True
    def update(self, screen, dt):
        self.draw(screen)
    def draw(self, screen):
        print("draw gameplay")
  
class Game:
    def __init__(self, **settings):
        self.__dict__.update(settings)
        self.done = False
    def setup_states(self, state_dict, start_state):
        self.state_dict = state_dict
        self.state_name = start_state
        self.state = self.state_dict[self.state_name]
        self.screen = None
    def flip_state(self):
        self.state.done = False
        previous,self.state_name = self.state_name, self.state.next
        self.state.cleanup()
        self.state = self.state_dict[self.state_name]
        self.state.startup()
        self.state.previous = previous
    def update(self, dt):
        if self.state.quit:
            self.done = True
        elif self.state.done:
            self.flip_state()
        self.state.update(self.screen, dt)
    def event_loop(self):
        # for event in pg.event.get():
        #     if event.type == pg.QUIT:
        #         self.done = True
        #     self.state.get_event(event)
        # dummy event
        event = None
        self.state.get_event(event)
    def main_game_loop(self):
        i = 0
        while not self.done and i < 5:
            i += 1
            self.event_loop()
            self.update(0.001)
            

In [8]:
settings = {
    'size':(600,400),
    'fps' :60
}
state_dict = {
    'menu': Menu(),
    'game': Gameplay()
}
  
game = Game(**settings)

game.setup_states(state_dict, 'menu')
game.main_game_loop()

cleaning up Menu state stuff
starting Game state stuff
draw gameplay
cleaning up Game state stuff
starting Menu state stuff
draw menu
cleaning up Menu state stuff
starting Game state stuff
draw gameplay
cleaning up Game state stuff
starting Menu state stuff
draw menu
cleaning up Menu state stuff
starting Game state stuff
draw gameplay


## Manual State Machine

In [4]:
from enum import Enum, auto

class State(Enum):
    OFF_HOOK = auto()
    CONNECTING = auto()
    CONNECTED = auto()
    ON_HOLD = auto()
    ON_HOOK = auto()

class Trigger(Enum):
    CALL_DIALED = auto()
    HUNG_UP = auto()
    CALL_CONNECTED = auto()
    PLACED_ON_HOLD = auto()
    TAKEN_OFF_HOLD = auto()
    LEFT_MESSAGE = auto()

In [22]:
from random import randint

rules = {
    State.OFF_HOOK: [(Trigger.CALL_DIALED, State.CONNECTING)],
    State.CONNECTING: [(Trigger.HUNG_UP, State.OFF_HOOK), (Trigger.CALL_CONNECTED, State.CONNECTED)],
    State.CONNECTED: [(Trigger.HUNG_UP, State.OFF_HOOK), (Trigger.LEFT_MESSAGE, State.ON_HOOK), (Trigger.PLACED_ON_HOLD, State.ON_HOLD)],
    State.ON_HOLD: [(Trigger.HUNG_UP, State.OFF_HOOK), (Trigger.TAKEN_OFF_HOLD, State.CONNECTED)],
    State.ON_HOOK: [(Trigger.HUNG_UP, State.OFF_HOOK)]
}

print_trigger_state = lambda s: [print(f"{i}: {item[0]}") for i, item in enumerate(rules[s])]

state = State.CONNECTING
end_state = State.OFF_HOOK

while state != end_state:
    print(f"Currently on {state}")
    print_trigger_state(state)

    choice = randint(0, len(rules[state])-1)
    print(f"choose trigger: {choice}")
    state = rules[state][choice][1]

print("ended")

Currently on State.CONNECTING
0: Trigger.HUNG_UP
1: Trigger.CALL_CONNECTED
choose trigger: 1
Currently on State.CONNECTED
0: Trigger.HUNG_UP
1: Trigger.LEFT_MESSAGE
2: Trigger.PLACED_ON_HOLD
choose trigger: 1
Currently on State.ON_HOOK
0: Trigger.HUNG_UP
choose trigger: 0
ended


## Use Library

In [36]:
# use pysm: https://pysm.readthedocs.io/en/latest/

from pysm import State, StateMachine, Event

on = State('on')
off = State('off')

sm = StateMachine('sm')
sm.add_state(on, initial=True)
sm.add_state(off)

sm.add_transition(on, off, events=['off'])
sm.add_transition(off, on, events=['on'])

sm.initialize()

def test():
    print(sm.state)
    sm.dispatch(Event('off'))
    print(sm.state)
    sm.dispatch(Event('on'))
    print(sm.state)
    sm.dispatch(Event('on'))
    print(sm.state)

In [37]:
test()

<State on (0x7fb9c9c43200)>
<State off (0x7fb9c9c432f0)>
<State on (0x7fb9c9c43200)>
<State on (0x7fb9c9c43200)>
